### 1  Importation

In [44]:
import pandas as pd
import numpy as np
import pickle


In [45]:
pip install tmdbv3api

Note: you may need to restart the kernel to use updated packages.


In [46]:
df_movies = pd.read_pickle("./data/fichier.pkl")
df_movies

,id,Titre,Original_title,Original_language,Production_companies,Production_countries,Release_date,Popularity,Budget,Overview,Runtime,Revenu,Spoken_languages,Tagline,Vote_average,Vote_count,Genres,Adult
0,2,Ariel,Ariel,fi,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",1988-10-21,20.997,0,Taisto Kasurinen is a Finnish coal miner whose...,73,0,"[[english_name, iso_639_1, name]]",,7.055,209,"[[id, name], [id, name], [id, name]]",False
1,3,Shadows in Paradise,Varjoja paratiisissa,fi,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",1986-10-17,15.827,0,"An episode in the life of Nikander, a garbage ...",74,0,"[[english_name, iso_639_1, name], [english_nam...",,7.245,204,"[[id, name], [id, name], [id, name]]",False
2,5,Four Rooms,Four Rooms,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name]]",1995-12-09,36.022,4000000,It's Ted the Bellhop's first night on the job....,98,4257354,"[[english_name, iso_639_1, name]]",Twelve outrageous guests. Four scandalous requ...,5.708,2268,"[[id, name], [id, name]]",False
3,6,Judgment Night,Judgment Night,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name]]",1993-10-15,19.026,21000000,"While racing to a boxing match, Frank, Mike, J...",109,12136938,"[[english_name, iso_639_1, name]]",Don't move. Don't whisper. Don't even breathe.,6.600,268,"[[id, name], [id, name], [id, name]]",False
4,8,Life in Loops (A Megacities RMX),Life in Loops (A Megacities RMX),en,"[[id, logo_path, name, origin_country]]","[[iso_3166_1, name]]",2006-01-01,2.033,42000,Timo Novotny labels his new project an experim...,80,0,"[[english_name, iso_639_1, name], [english_nam...",A Megacities remix.,7.738,21,"[[id, name]]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,493,Shaft's Big Score!,Shaft's Big Score!,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name]]",1972-06-08,5.672,1978000,John Shaft is back as the lady-loved black det...,104,10000000,"[[english_name, iso_639_1, name]]",Last time he was nice. This time he's ice!,6.186,43,"[[id, name], [id, name], [id, name], [id, name]]",False
384,494,Shaft in Africa,Shaft in Africa,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name]]",1973-06-14,5.954,2142000,Detective John Shaft travels incognito to Ethi...,112,1458000,"[[english_name, iso_639_1, name]]",Shaft is stickin' it...all the way.,6.000,41,"[[id, name], [id, name], [id, name], [id, name...",False
385,496,Borat: Cultural Learnings of America for Make ...,Borat: Cultural Learnings of America for Make ...,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name], [iso_3166_1, name]]",2006-11-01,18.973,18000000,Kazakh journalist Borat Sagdiyev travels to Am...,84,262552893,"[[english_name, iso_639_1, name], [english_nam...","Come to Kazakhstan, it's nice!",6.739,4592,"[[id, name]]",False
386,497,The Green Mile,The Green Mile,en,"[[id, logo_path, name, origin_country], [id, l...","[[iso_3166_1, name]]",1999-12-10,74.257,60000000,A supernatural tale set on death row in a Sout...,189,286801374,"[[english_name, iso_639_1, name], [english_nam...",Miracles do happen.,8.509,14741,"[[id, name], [id, name], [id, name]]",False


### 2 Analyse descriptive

In [17]:
# Cleaning Genres :
def traitement_genre(genre):
    genres = []
    for type_genres in genre:
        genres.append(type_genres["name"])    
    return genres
df["genres"] = df["Genres"].apply(traitement_genre)







TypeError: string indices must be integers

### 3 Modélisation